#Install Dependencies and Import libraries


In [45]:
!pip install flask flask-cors pyngrok google-generativeai

In [46]:
import google.generativeai as genai
from google.colab import userdata #for secrets
import json
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import threading
import os
from pyngrok import ngrok
import re

#Configure Gemini API Key

In [47]:
try:
  api_key = userdata.get('GEMINI_API_KEY')
  genai.configure(api_key=api_key)
  print("Gemini API Key configured successfully")
except userdata.SecretNotFoundError:
  print("ERROR: Gemini API Key not found in userdata. Please add it in Colab Secrets manager")
  api_key = None
except Exception as e:
  print(f"An Error has occured.... ERROR: {e}")
  api_key = None

Gemini API Key configured successfully


In [48]:
llm_model = None
if api_key:
  try:
    llm_model = genai.GenerativeModel('gemini-1.5-flash')
    print("Gemini model initialized")
  except Exception as e:
    print(f"Error initializing gemini model: {e}")
else:
  print("Skipping model initilization due to missing API key.")

!mkdir -p game_files

Gemini model initialized


Configure Flask App

In [49]:
app = Flask(__name__,static_folder='game_files')
CORS(app)

Define LLM API Endpoint

In [50]:
'''
@app.route("/llm/move",methods=["POST"])
def llm_move():
  data = request.json
  if not llm_model:
    return jsonify({"error": "LLM model not initialized."}),500

  units = data.get("units",[])
  instructions = data.get("instructions","")
  elevation_map_info = data.get("elevationMapInfo","")

  prompt = f"""
You are playing a strategy game where you control military units on a map with varying elevations.
Your primary goal is to move units efficiently, minimizing fuel consumption, especially when moving across significant elevation changes.

**Game Context:**
- **Units (ID, Name, Current Position, Current Fuel):**
  {json.dumps(units, indent=2)}

- **Elevation Map (`elevationMapInfo`):** This is a 2D array representing the elevation at each (x, y) coordinate on the map.
  The map coordinates correspond directly to the array indices. For example, `elevationMapInfo[y][x]` gives the elevation at (x, y).
  Elevations can range from approximately -10 to 10 (mapped from 0-255 grayscale values where 0 is -10 and 255 is 10).
  A sample of the elevation map (first few rows):
  {json.dumps(elevation_map_info[:min(5, len(elevation_map_info))], indent=2)}
  (The full map is provided)

- **Fuel Consumption Rules:**
  - Moving across flat terrain (no elevation change) consumes less fuel.
  - Moving uphill or downhill (any change in elevation) consumes significantly more fuel.
  - The greater the *absolute difference* in elevation between two adjacent points on the path, the higher the fuel cost for that segment of movement.
  - Fuel is also consumed based on the distance moved.

**User Instruction:**
\"\"\"{instruction}\"\"\"

**Your Task:**
Based on the user's instruction, the current unit states, and the elevation map, propose a move plan for the units.
For each unit, you must decide its target `x` and `y` coordinates.
Your moves **must prioritize reaching the target with the least amount of fuel possible**, carefully considering the elevation data provided.
If a unit needs to clear its current movement queue and move directly to the target, set "queue" to `false`. If it should add the new target to its existing movement queue, set "queue" to `true`.

**Output Format:**
Respond ONLY with a JSON list of move commands.
Each command must be a JSON object like this:
{{"unitId": <number>, "x": <number>, "y": <number>, "queue": <true/false>}}

Example Response (if moving unit 1 to 3,4 directly, and unit 2 to 5,6 by adding to queue):
[
  {{"unitId": 1, "x": 3, "y": 4, "queue": false}},
  {{"unitId": 2, "x": 5, "y": 6, "queue": true}}
]
    """
  print("--- LLM Prompt ---")
  print(prompt)
  print("--- End of LLM Prompt ---")
  try:
      response = llm_model.generate_content(prompt)
      raw = response.text.strip()
      print("--- LLM Response ---")
      print(raw)
      try:
          move_plan = json.loads(raw) # string to python dictionary
          if isinstance(move_plan, list):
              return jsonify({"plan": move_plan})
          else:
              return jsonify({"error": "LLM did not return a list", "raw": raw}), 500
      except json.JSONDecodeError as e:
          return jsonify({"error": "Could not parse LLM output (Invalid JSON)", "details": str(e), "raw": raw}), 500
      except Exception as e:
          return jsonify({"error": "Error parsing LLM output", "details": str(e), "raw": raw}), 500
  except Exception as e:
        return jsonify({"error": f"Error calling LLM API: {e}"}), 500
'''

'\n@app.route("/llm/move",methods=["POST"])\ndef llm_move():\n  data = request.json\n  if not llm_model:\n    return jsonify({"error": "LLM model not initialized."}),500\n\n  units = data.get("units",[])\n  instructions = data.get("instructions","")\n  elevation_map_info = data.get("elevationMapInfo","")\n\n  prompt = f"""\nYou are playing a strategy game where you control military units on a map with varying elevations.\nYour primary goal is to move units efficiently, minimizing fuel consumption, especially when moving across significant elevation changes.\n\n**Game Context:**\n- **Units (ID, Name, Current Position, Current Fuel):**\n  {json.dumps(units, indent=2)}\n\n- **Elevation Map (`elevationMapInfo`):** This is a 2D array representing the elevation at each (x, y) coordinate on the map.\n  The map coordinates correspond directly to the array indices. For example, `elevationMapInfo[y][x]` gives the elevation at (x, y).\n  Elevations can range from approximately -10 to 10 (mapped f

In [51]:
@app.route("/llm/move", methods=["POST"])
def llm_move():
    data = request.json
    if not llm_model:
        return jsonify({"error": "LLM model not initialized."}), 500

    units = data.get("units", [])
    instruction = data.get("instruction", "")
    elevation_map_info = data.get("elevationMapInfo", [])
    num_test_moves = data.get("numTestMoves", 5)

    print("\n--- Received Data from Client ---")
    print(f"Instruction: {instruction}")
    print(f"Number of test moves requested: {num_test_moves}")
    print(f"Units data received: {len(units)} units")
    print(f"Elevation map data received: {len(elevation_map_info)} rows")

    prompt = f"""
You are an intelligent tactical AI controlling military units in a strategy game.
Your objective is to move units efficiently across a terrain with varying elevations, minimizing fuel consumption.

**Game State Information:**

1.  **Units:**
    {json.dumps(units, indent=2)}
    Each unit has: `unitId`, `unitName`, `position` (x, y), and `currentFuel`.

2.  **Elevation Map (`elevationMapInfo`):**
    This is a 2D array (list of lists) where `elevationMapInfo[y][x]` gives the elevation at map coordinate (x, y).
    The map dimensions are {len(elevation_map_info[0]) if elevation_map_info and len(elevation_map_info) > 0 else 'N/A'} (width) by {len(elevation_map_info) if elevation_map_info else 'N/A'} (height).
    Elevations are integers ranging from approximately -10 (lowest) to 10 (highest).
    Moving between coordinates with different elevations consumes more fuel. The greater the absolute difference in elevation, the higher the fuel cost for that segment.
    Moving 1 unit distance on flat terrain (no elevation change) has a base fuel cost.
    Moving 1 unit distance with an elevation change of 'E' costs roughly `base_fuel_cost * (1 + abs(E) * 0.2)`.
    (Note: The game's `estimateFuelUsageTo` function uses `mapNumber(Math.abs(slope), 0, 10, 1, 0.01)` for speed and `1 + Math.abs(slope) * 0.2` for fuel factor. You should use the `1 + abs(E) * 0.2` logic for fuel cost consideration.)

    A small sample of the elevation map (first {min(5, len(elevation_map_info))} rows for context):
    {json.dumps(elevation_map_info[:min(5, len(elevation_map_info))], indent=2)}
    The full map data is available for your calculations.

**Your Task:**

Based on the **User Instruction** below, for each unit:
1.  **Generate {num_test_moves} distinct potential movement paths.** Each path should be a sequence of 1 to 5 waypoints (x, y coordinates). These paths should aim towards the general direction implied by the instruction (e.g., nearest enemy, or a random distant point if no specific target is given).
2.  **For each generated path, calculate its total estimated fuel consumption.** To do this, iterate through the waypoints in the path. For each segment between two consecutive waypoints (A to B):
    * Calculate the distance between A and B.
    * For each small step (e.g., 1 pixel) along the segment from A to B:
        * Determine the current elevation and the elevation at the next pixel using `elevationMapInfo`.
        * Calculate the absolute elevation change (slope) between these two pixels.
        * Apply the fuel cost factor: `(1 + abs(slope) * 0.2)`.
        * Accumulate fuel cost for this small step.
    * Sum up the fuel costs for all steps in the segment, and then for all segments in the path.
    * **Simplified Fuel Calculation Guidance for LLM:** For simplicity in LLM reasoning, you can approximate fuel cost for a segment from `(x1, y1)` to `(x2, y2)` by considering the elevation difference between `elevationMapInfo[y1][x1]` and `elevationMapInfo[y2][x2]` (if you only want to consider start and end of segment), and the Euclidean distance. A more detailed per-pixel simulation is ideal but might be too complex for a single LLM turn. Focus on the *overall* elevation change impact along the path.

3.  **Select the path with the lowest estimated fuel consumption** for that unit.
4.  **Output the selected optimal path for each unit.**

**User Instruction:**
\"\"\"{instruction}\"\"\"

**Output Format:**
Respond ONLY with a JSON list of `UnitMovePlan` objects. Each `UnitMovePlan` must contain:
-   `unitId`: The ID of the unit.
-   `path`: An array of `{{\"x\": number, \"y\": number}}` objects, representing the ordered waypoints of the chosen path. # <--- THIS LINE IS THE KEY FIX

Example Response:
[
  {{"unitId": 1, "path": [{{"x": 100, "y": 100}}, {{"x": 150, "y": 120}}]}},
  {{"unitId": 2, "path": [{{"x": 200, "y": 300}}, {{"x": 205, "y": 310}}, {{"x": 210, "y": 320}}]}}
]
    """

    print("\n--- LLM Prompt ---")
    print(prompt)

    try:
        response = llm_model.generate_content(
            prompt,
            generation_config={
                "response_mime_type": "application/json",
                "response_schema": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "unitId": {"type": "number"},
                            "path": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "x": {"type": "number"},
                                        "y": {"type": "number"}
                                    },
                                    "required": ["x", "y"]
                                }
                            }
                        },
                        "required": ["unitId", "path"]
                    }
                }
            }
        )

        raw_response_text = response.text.strip()

        print("\n--- Raw LLM Response ---")
        print(raw_response_text)

        try:
            move_plan = json.loads(raw_response_text)

            if isinstance(move_plan, list):
                print("\n--- Parsed LLM Plan ---")
                print(json.dumps(move_plan, indent=2))
                return jsonify({"plan": move_plan})
            else:
                print(f"ERROR: LLM did not return a JSON list. Raw: {raw_response_text}")
                return jsonify({"error": "LLM did not return a list as expected", "raw": raw_response_text}), 500

        except json.JSONDecodeError as e:
            print(f"ERROR: Could not parse LLM output as JSON. Details: {e}. Raw: {raw_response_text}")
            return jsonify({"error": "LLM response was not valid JSON", "details": str(e), "raw": raw_response_text}), 500
        except Exception as e:
            print(f"ERROR: An unexpected error occurred during LLM response parsing. Details: {e}. Raw: {raw_response_text}")
            return jsonify({"error": "Error parsing LLM output", "details": str(e), "raw": raw_response_text}), 500

    except Exception as e:
        print(f"ERROR: Error calling LLM API. Details: {e}")
        return jsonify({"error": f"Error calling LLM API: {e}"}), 500

In [52]:
@app.route('/')
def index():
  #Serve index.html from the 'game_files' directory
  return send_from_directory('game_files','index.html')

@app.route('/<path:filename>')
def serve_static(filename):
  #Serve the other files
  return send_from_directory('game_files',filename)



In [53]:
def run_flask():
    # Needs to run on port 8880 (or any available port) for Ngrok to tunnel to
    print("Starting Flask server...")
    # Use host='0.0.0.0' to make it accessible from outside the Colab environment
    app.run(host='0.0.0.0', port=8880, debug=False)

def start_ngrok():
    try:
        # Terminate existing tunnels if any, to avoid conflicts
        ngrok.kill()

        # Get ngrok auth token from Colab secrets if you have one (recommended)
        try:
            ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
            ngrok.set_auth_token(ngrok_auth_token)
            print("Ngrok auth token set.")
        except userdata.SecretNotFoundError:
            print("INFO: NGROK_AUTH_TOKEN secret not found. Using ngrok without auth token (temporary URL).")
        except Exception as e:
            print(f"Error setting ngrok auth token: {e}")

        # Start an HTTP tunnel on the same port Flask is running on (8880)
        public_url = ngrok.connect(8880, "http")
        print(f" * ngrok tunnel available at: {public_url}")
        return public_url
    except Exception as e:
        print(f"Error starting ngrok: {e}")
        return None

In [ ]:
print("\n--- Starting Server Setup ---")

# Start ngrok first to get the public URL
public_url = start_ngrok()

if public_url:
    print(f"\n--- Server Setup Complete ---")
    print(f"Access your game POC at: {public_url}")
    print(f"LLM endpoint will be at: {public_url}/llm/move") # The full URL for your game to call
else:
    print("\n--- Server Setup Failed ---")
    print("Failed to start ngrok tunnel. Server might be running but not accessible publicly.")
    print("Please ensure your NGROK_AUTH_TOKEN is correctly set in Colab secrets (🔒 icon).")

print("\n--- Starting Flask Server (Foreground - Blocking) ---")
print(f"Flask is now running and listening on port 8880.")
print("Logs from Flask requests will appear directly below this cell.")
print("!!! The Colab cell will appear 'busy'. To stop the server, you MUST interrupt the kernel (Runtime -> Interrupt execution) !!!")
try:
    # Run Flask on 0.0.0.0 to accept connections from ngrok
    run_flask() # This will block the cell until the server is stopped
except KeyboardInterrupt:
    print("\n--- Flask server stopped (KeyboardInterrupt) ---")
    ngrok.kill() # Ensure ngrok tunnel is also terminated
finally:
    print("--- Flask server has been shut down ---")


--- Starting Server Setup ---
Ngrok auth token set.
 * ngrok tunnel available at: NgrokTunnel: "https://b374-34-127-70-181.ngrok-free.app" -> "http://localhost:8880"

--- Server Setup Complete ---
Access your game POC at: NgrokTunnel: "https://b374-34-127-70-181.ngrok-free.app" -> "http://localhost:8880"
LLM endpoint will be at: NgrokTunnel: "https://b374-34-127-70-181.ngrok-free.app" -> "http://localhost:8880"/llm/move

--- Starting Flask Server (Foreground - Blocking) ---
Flask is now running and listening on port 8880.
Logs from Flask requests will appear directly below this cell.
!!! The Colab cell will appear 'busy'. To stop the server, you MUST interrupt the kernel (Runtime -> Interrupt execution) !!!
Starting Flask server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8880
 * Running on http://172.28.0.12:8880
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Jul/2025 19:36:42] "OPTIONS /llm/move HTTP/1.1" 200 -



--- Received Data from Client ---
Instruction: 
    For each unit, generate 5 distinct potential movement paths (sequences of waypoints) Towards (500,500)
    Each path should consist of 1 to 5 waypoints.
    For each generated path, estimate its total fuel cost by considering the elevation changes between consecutive waypoints.
    The fuel cost increases significantly with the absolute difference in elevation between adjacent points.
    After evaluating all 5 paths for a unit, select the path that has the lowest estimated fuel consumption.
    Then, provide this selected optimal path for each unit.
    
Number of test moves requested: 5
Units data received: 2 units
Elevation map data received: 1280 rows

--- LLM Prompt ---

You are an intelligent tactical AI controlling military units in a strategy game.
Your objective is to move units efficiently across a terrain with varying elevations, minimizing fuel consumption.

**Game State Information:**

1.  **Units:**
    [
  {
    "unitI

INFO:werkzeug:127.0.0.1 - - [05/Jul/2025 19:36:46] "POST /llm/move HTTP/1.1" 200 -


Streaming output truncated to the last 5000 lines.
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    3,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    4,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    6,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    5,
    4,
    4,
    4,
    4,
    

INFO:werkzeug:127.0.0.1 - - [05/Jul/2025 19:36:49] "POST /llm/move HTTP/1.1" 200 -



--- Raw LLM Response ---
[{"path":[{"x":50,"y":50},{"x":250,"y":250},{"x":500,"y":500}],"unitId":0},{"path":[{"x":100,"y":100},{"x":300,"y":300},{"x":500,"y":500}],"unitId":1}]

--- Parsed LLM Plan ---
[
  {
    "path": [
      {
        "x": 50,
        "y": 50
      },
      {
        "x": 250,
        "y": 250
      },
      {
        "x": 500,
        "y": 500
      }
    ],
    "unitId": 0
  },
  {
    "path": [
      {
        "x": 100,
        "y": 100
      },
      {
        "x": 300,
        "y": 300
      },
      {
        "x": 500,
        "y": 500
      }
    ],
    "unitId": 1
  }
]
